In [1]:
import pandas as pd 
import requests
import json

In [10]:
df_original = pd.read_csv("43_abby.csv", sep=';').fillna(value=0).rename(columns={"Unnamed: 0": "context"})
df_original = df_original.iloc[:, [j for j, c in enumerate(df_original.columns) if j != 0]]

df_original.head()

,context,eng-ru,eng-ru-check,ru-fr,ru-it,протяженный,точечный,1D,2D,3D,нет увенч.,увенч.,1D жесткий,1D мягкий
0,wall,стена,стена,mur,muro,+,0,+,0,0,0,0,+,0
1,gorge,горло,ущелье,gorge,gola,+,0,+,0,0,0,0,+,0
2,ridge,гребень,гребень,peigne,pettine,+,0,+,0,0,0,0,+,0
3,wave,вал,волна,mur,baluardo,+,0,+,0,0,0,0,0,+
4,waterfall,водопад,водопад,chute,cascata,+,0,+,0,0,0,0,0,+


In [11]:
list(df_original['context'].values)

['wall',
 'gorge',
 'ridge',
 'wave',
 'waterfall',
 'building',
 'skull',
 'head',
 'house',
 'arch',
 'volcano',
 'hill',
 'mountain',
 'mound',
 'cliff',
 'rock',
 'helmet',
 'peak',
 'slope',
 'tree',
 'tower',
 'pyramid',
 'iceberg',
 'mast',
 'pole',
 'stem',
 'shaft',
 'temple',
 'church',
 'belltower',
 'cathedral',
 'asparagus']

In [4]:
URL_AUTH = 'https://developers.lingvolive.com/api/v1.1/authenticate'
URL_TRANSLATE = 'https://developers.lingvolive.com/api/v1/Minicard'
KEY = 'ZjZmNTA3ODctZmNlMS00Njk2LTgyNWYtOTExYTRmNjA0NjhhOmJiOTZiYWRmMDBjZTRlYWNiODVkNGVlMDRhMDViZmIw'

def get_auth_token(key: str, url_auth: str) -> str:
    headers_auth = {'Authorization': 'Basic ' + key}
    auth = requests.post(url=url_auth, headers=headers_auth)
    if auth.status_code == 200:
        cur_token = auth.text
        return cur_token
    else:
        print('Error - ' + str(auth.status_code))
        return ''

def get_a_word_translation(cur_token: str, url_tr: str, word: str, index1, index2) -> str:
    headers_translate = {
        'Authorization': 'Bearer ' + cur_token
    }
    params = {
        'text': word,
        'srcLang': index1,
        'dstLang': index2
    }
    req = requests.get(
        url_tr, headers=headers_translate, params=params)
    if req.status_code == 200:
        res = req.json()
        try:
            value = res['Translation']['Translation'].split()[0].replace(',','').replace(';','').replace(':','').replace('.','')
            return value
        except TypeError:
            if res == 'Incoming request rate exceeded for 50000 chars per day pricing tier':
                print('Error - Incoming request rate exceeded for 50000 chars per day pricing tier')
                return res
            else:
                return 'No translation available'
    else:
        print('Error!' + str(req.status_code))
        

In [5]:
token = get_auth_token(key=KEY, url_auth=URL_AUTH)
not_translated_words = list(df_original['context'].values)
translated_words = []


for en_word in not_translated_words:
    ru_translation = get_a_word_translation(cur_token=token, url_tr=URL_TRANSLATE, word=en_word,index1=1033,index2=1049)
    if ru_translation == 'Incoming request rate exceeded for 50000 chars per day pricing tier':
        break
    translated_words.append(ru_translation)
print(not_translated_words, len(not_translated_words))
print(translated_words, len(translated_words))




Error!404
['wall', 'gorge', 'ridge', 'wave', 'waterfall', 'building', 'skull', 'head', 'house', 'arch', 'volcano', 'hill', 'mountain', 'mound', 'cliff', 'rock', 'helmet', 'peak', 'slope', 'tree', 'tower', 'pyramid', 'iceberg', 'mast', 'pole', 'stem', 'shaft', 'temple', 'church', 'belltower', 'cathedral', 'asparagus'] 32
['стена', 'горло', 'гребень', 'вал', 'водопад', 'здание', 'череп', 'голова', 'дом', 'арка', 'вулкан', 'возвышение', 'гора', 'насыпь', 'отвесная', 'скала', 'защитный', 'пик', 'наклон', 'дерево', 'башня', 'пирамида', 'айсберг', 'мачта', 'столб', 'ствол', 'древко', 'храм', 'церковь', None, 'кафедральный', 'спаржа'] 32


In [12]:
df_original = pd.read_csv("43_abby.csv", sep=';').fillna(value=0).rename(columns={"Unnamed: 0": "context"})
df_original = df_original.iloc[:, [j for j, c in enumerate(df_original.columns) if j != 0]]
df_original.insert(1, "eng-ru",translated_words, True) 

ValueError: Length of values does not match length of index

In [9]:
df_original.head(5)

,context,context,eng-ru,eng-ru-check,ru-fr,ru-it,протяженный,точечный,1D,2D,3D,нет увенч.,увенч.,1D жесткий,1D мягкий
0,0,wall,стена,стена,mur,muro,+,0,+,0,0,0,0,+,0
1,1,gorge,горло,ущелье,gorge,gola,+,0,+,0,0,0,0,+,0
2,2,ridge,гребень,гребень,peigne,pettine,+,0,+,0,0,0,0,+,0
3,3,wave,вал,волна,mur,baluardo,+,0,+,0,0,0,0,0,+
4,4,waterfall,водопад,водопад,chute,cascata,+,0,+,0,0,0,0,0,+


In [ ]:
not_translated_words = list(df_original['eng-ru-check'].values)
translated_words = []

for en_word in not_translated_words:
    ru_translation = get_a_word_translation(cur_token=token, url_tr=URL_TRANSLATE, word=en_word,index1=1049,index2=1040)
    if ru_translation == 'Incoming request rate exceeded for 50000 chars per day pricing tier':
        break
    translated_words.append(ru_translation)
    
df_original.insert(2, "ru-it",translated_words, True) 

translated_words = []

for en_word in not_translated_words:
    ru_translation = get_a_word_translation(cur_token=token, url_tr=URL_TRANSLATE, word=en_word,index1=1049,index2=1036)
    if ru_translation == 'Incoming request rate exceeded for 50000 chars per day pricing tier':
        break
    translated_words.append(ru_translation)
    
df_original.insert(2, "ru-fr",translated_words, True) 

In [7]:
df_original

NameError: name 'time' is not defined

In [41]:
df_original

,context,eng-ru,ru-fr,ru-it,eng-ru-check,ru-fr,ru-it,протяженный,точечный,1D,2D,3D,нет увенч.,увенч.,1D жесткий,1D мягкий
0,wall,стена,None,muro,стена,mur,muro,+,0,+,0,0,0,0,+,0
1,gorge,горло,None,n,ущелье,gorge,gola,+,0,+,0,0,0,0,+,0
2,ridge,гребень,None,None,гребень,peigne,pettine,+,0,+,0,0,0,0,+,0
3,wave,вал,vague,onda,волна,mur,baluardo,+,0,+,0,0,0,0,0,+
4,waterfall,водопад,None,None,водопад,chute,cascata,+,0,+,0,0,0,0,0,+
5,building,здание,None,None,здание,édifice,edificio,+,0,0,+,0,0,0,0,0
6,skull,череп,None,None,череп,crâne,cranio,+,0,0,+,0,0,0,0,0
7,head,голова,None,None,голова,tête,testa,+,0,0,0,+,0,0,0,0
8,house,дом,None,None,дом,maison,casa,+,0,0,0,+,0,0,0,0
9,arch,арка,None,None,арка,arc,arco,+,0,0,0,+,0,0,0,0


In [47]:
df_original.to_csv('42_updated.csv')
